In [8]:
'''
0: boundary and wall
1: path can be moved
2: player
3: box
4: storage

Known
[[0 0 0 0 0 0]
 [0 4 3 1 2 0]
 [0 0 0 0 0 0]]
Transform               ## Remove storage
[[0 0 0 0 0 0]
 [0 1 3 1 2 0
 [0 0 0 0 0 0]]
result                 ## Only show storage 
                          when coordinates of boxes match coordinates of storage: complete
[[0 0 0 0 0 0]          
 [0 4 1 1 1 0]
 [0 0 0 0 0 0]]
'''

'\n0: boundary and wall\n1: path can be moved\n2: player\n3: box\n4: storage\n\nKnown\n[[0 0 0 0 0 0]\n [0 4 3 1 2 0]\n [0 0 0 0 0 0]]\nTransform               ## Remove storage\n[[0 0 0 0 0 0]\n [0 1 3 1 2 0\n [0 0 0 0 0 0]]\nresult                 ## Only show storage \n                          when coordinates of boxes match coordinates of storage: complete\n[[0 0 0 0 0 0]          \n [0 4 1 1 1 0]\n [0 0 0 0 0 0]]\n'

In [9]:
import numpy as np
import copy
import time

def person_position(P_1):
    
    # search the position of player
    for i in range(len(P_1)): #length
        for j in range(len(P_1[0])): #width
            if P_1[i][j] == 2 or P_1[i][j] == 6:
                return ([i,j])

def box_position(P_1):
    
    # search the position of box
    box = []
    for i in range(len(P_1)):
        for j in range(len(P_1[0])):
            if P_1[i][j] == 3:
                box.append([i,j])
    return (box)


def target_position(P_1):
    
    # search the position of target
    target = []
    for i in range(len(P_1)):
        for j in range(len(P_1[0])):
            if P_1[i][j] == 4:
                target.append([i,j])
    return (target)

def transform_P1(P):
    
    ## initial state, ignore storage
    P_1=copy.deepcopy(P)
    for i in range(len(P)):
        for j in range(len(P[0])):
            if P_1[i][j]==4:
                P_1[i][j]=1
    return P_1

def transform_Pend(P):
    
    ## final state，ignore person and boxes all in storage
    P_end=copy.deepcopy(P)
    for i in range(len(P)):
        for j in range(len(P[0])):
            if P_end[i][j]==2 or P_end[i][j]==3:
                P_end[i][j]=1
    return P_end

def arrayin(P1,P2):
    for i in range(len(P2)): 
        if (P1-P2[i]).any():   # P1 - P2[i],只要有一个元素>0，则.any()返回True，if成立，进入下一圈循环，
                                # 如果P1 - P2[i] = 0, 即P1和P2[i]相同，则.any()返回False，整体结果返回True
                                # 如果没有P2[i]和P1相同，则整体结果返回False
            continue             
        else:
            return True
    return False

# a = np.array([1,2,3])
# b = np.array([1,1,3])
# print((a-b).any())
# True

# a = np.array([1,2,3])
# b = np.array([1,2,3])
# print((a-b).any())
# False

############################################################我的改动
dirs=[[0,-1],[-1,0],[0,1],[1,0]]## four directions 'L''U''R''D'


def is_OK(P_1,P_end):
    ## check if current state match final state
    for i in range(len(P_1)):
        for j in range(len(P_1[0])):
            if P_1[i][j]!=3 and  P_end[i][j]==4:
                return False   # 没推到位置
    return True # 所有的坐标，3都在相应end4的位置

def best_move(pathfinal):
    moves = ''
    for i in range(len(pathfinal)):
        if pathfinal[i]==[0,-1]:
            moves+='L'
        if pathfinal[i]==[-1,0]:
            moves+='U'
        if pathfinal[i]==[0,1]:
            moves+="R"
        if pathfinal[i]==[1,0]:
            moves+='D'
    return moves
    

In [16]:
def BFS(P_1,person):
    
    states=[[P_1,[],person]]
    
    while(len(states)>0):
        #print('=======================')
        #print(len(states))
        #print('========================')
        print(len(states))
        global pathfinal
        P=np.array(states[0][0])##initial status
        path=states[0][1]
        person=states[0][2] 
        if(len(path)>iterate_max):                                 ##deal with too long loop
            break
        
        if(is_OK(P,P_end)):   # 所有箱子都推到位，返回true；否则，返回false
            if len(pathfinal)==0:
                pathfinal=path
                continue                                                ##check if ok
            elif len(path)<len(pathfinal):
                pathfinal=path
                continue
        states=states[1:]  ##explore the current status,update the frontier
        for i,dir in enumerate(dirs):
            global visited
            neg=[person[0]+dir[0],person[1]+dir[1]]##the neighbour
            nneg=[person[0]+2*dir[0],person[1]+2*dir[1]]   ##the neighbour of the neighbour
            
            #if not (neg[0]>=0 and neg[0]<len(P) and neg[1]>=0 and neg[1]<len(P[0])):
            if not (neg[0]>0 and neg[0]<(len(P)-1) and neg[1]>0 and neg[1]<(len(P[0])-1)):
                continue     # 触碰边界
            if P[neg[0]][neg[1]]==1:           ## only the person move
                Pnew=copy.deepcopy(P)
                Pnew[neg[0]][neg[1]]=2
                Pnew[person[0]][person[1]]=1 # 移动一步，更新地图
                if not arrayin(Pnew,visited):## check if visited
                    visitedlist=visited.tolist()
                    visitedlist.append(Pnew.tolist())
                    visited=np.array(visitedlist)
                    states.append([Pnew,path+[dir],neg])  ##add new status in frontier
            elif P[neg[0]][neg[1]]==3 and P[nneg[0]][nneg[1]]==1:
                Pnew=copy.deepcopy(P)
                Pnew[neg[0]][neg[1]]=2
                Pnew[person[0]][person[1]]=1
                Pnew[nneg[0]][nneg[1]]=3
                if not arrayin(Pnew,visited):                                            ## check if visited
                    visitedlist=visited.tolist()
                    visitedlist.append(Pnew.tolist())
                    visited=np.array(visitedlist)
                    states.append([Pnew,path+[dir],neg])                           ##add new status in frontier

In [18]:
start =time.clock()

#P = np.array([[0,0,0,0,0,0,0,0],[0,4,1,0,1,1,1,0],[0,1,1,3,1,1,1,0],[0,1,1,1,0,1,0,0],[0,0,1,0,1,3,4,0],[0,1,1,1,3,1,1,0],[0,1,1,4,0,1,2,0],[0,0,0,0,0,0,0,0]])
P=np.array([[0,0,0,0,0,0,0],[0,1,1,1,1,2,0],[0,4,3,1,3,4,0],[0,1,1,1,1,1,0],[0,0,0,0,0,0,0]])
print(P)

states=[] ## frontier, record states to explore

iterate_max=1000 ## death loop maximum

person=person_position(P)

P_1=transform_P1(P)

visited=np.array([P_1])## record visited states

P_end=transform_Pend(P)

pathfinal=[]

print(P_1)
print(P_end)

BFS(P_1,person)

print(best_move(pathfinal))

end = time.clock()
print('Running time: %s Seconds'%(end-start))

E:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


[[0 0 0 0 0 0 0]
 [0 1 1 1 1 2 0]
 [0 4 3 1 3 4 0]
 [0 1 1 1 1 1 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 1 1 1 1 2 0]
 [0 1 3 1 3 1 0]
 [0 1 1 1 1 1 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 1 1 1 1 1 0]
 [0 4 1 1 1 4 0]
 [0 1 1 1 1 1 0]
 [0 0 0 0 0 0 0]]
1
2
3
4
5
7
9
9
10
12
14
14
14
15
15
15
15
15
17
19
21
21
23
23
24
23
23
24
23
22
23
25
26
27
27
28
29
30
30
32
32
32
32
33
34
34
35
36
37
39
40
41
42
44
46
47
46
48
47
46
46
46
48
49
49
49
49
51
51
51
52
52
52
52
52
54
54
54
54
54
54
54
56
55
57
58
58
60
61
60
60
61
63
63
62
63
65
67
67
69
70
70
69
68
69
69
69
71
72
74
74
74
75
76
76
76
77
77
77
78
79
79
79
79
80
80
81
82
84
83
84
84
84
85
86
86
87
87
87
88
90
89
90
91
90
91
93
93
93
92
91
92
92
93
93
93
94
95
95
96
97
97
98
98
97
98
98
98
98
99
99
100
99
98
99
100
100
100
102
102
103
104
103
103
103
103
104
104
106
105
104
104
103
102
102
102
103
102
104
103
103
103
103
102
104
103
104
103
103
102
102
103
103
103
104
103
103
104
104
106
105
107
106
106
105
105
105
104
104
104
104


E:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [13]:
a=[[1,2]]
b=[[1,2],[3,4]]
b=a
print(b)

[[1, 2]]
